In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
# base_df = pd.read_csv("../no_repeated.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
base_df = pd.read_csv("../BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",usecols=['SessionID', 'TIMESTAMP', 'URL_FILE', ])
base_df.rename(columns={'URL_FILE':'Activity'}, inplace=True)
base_df["TIMESTAMP"] = pd.to_datetime(base_df["TIMESTAMP"], infer_datetime_format=True)
base_df = base_df.sort_values(["SessionID", "TIMESTAMP"])
base_df.head()

,SessionID,TIMESTAMP,Activity
3273278,46,2015-11-06 08:07:22.780,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken
5333642,46,2015-11-06 08:07:40.767,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...
3733243,46,2015-11-06 08:07:51.390,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...
5904405,46,2015-11-06 08:08:06.003,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...
5573282,46,2015-11-06 08:08:19.343,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...


In [3]:
# Helper functions
def create_df_for_pivot(_df: pd.DataFrame):
  df = _df.copy()
  #Create consecutive column with the consecutive activity and count how many time each pair occurs
  df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

  #Count all the pairs and safe in new DataFrame
  df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_1'],))), index=["value"]).T.reset_index()

  df_for_pivot_1.fillna("end_session", inplace=True)
  return df_for_pivot_1
  
  
def make_pivot(_df, index_names, column_names):
  df = _df.copy()
  """
  Functions that takes in a DataFrame and returns a pivot table with all the chances
  
  Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
  """
  df_chances = df.pivot_table(index=index_names, columns=column_names, values='value')
  
  #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
  df_chances["total_row_count"] = df_chances.sum(axis=1)
  df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
  
  df_chances.fillna(0, inplace=True)
  df_chances.drop("total_row_count", axis=1, inplace=True)

  return df_chances

def calculate_paths_in_sessions(_df:pd.DataFrame,paths:list[str]) -> tuple[int,int]:
  # this function calculates in how many sessions a path is present
  # a path is a series of transitions
  # a transition is two consecutive activities where the 2 activities are not the same
  # we return a tuple where the first value is the total amount of times the path is found and
  # the second value is the total amount of sessions where the path is found
  # group the dataframe by session id
  df = _df.copy()
  # create a new column that contains the consecutive activity
  df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)
  # if the consecutive activity is the same as the current activity, remove the row
  # this is done because we only want to count transitions
  df = df[df["Activity"] != df["Consecutive_1"]]
  # remove the consecutive activity column
  df = df.drop("Consecutive_1", axis=1)
  # get the length of the path
  len_path = len(paths)
  # create a column for len_path - 1 consecutive activities
  df = df.assign(**{f"Activity_{i}": df.groupby("SessionID")["Activity"].shift(periods=-i) for i in range(1, len_path)})
  # change name of Activiy column to Activity_0
  df.rename(columns={'Activity':'Activity_0'}, inplace=True)
  # create a column "Found" that contains True if the path is found in the session
  df['Found'] = df.apply(lambda row: all(row[f"Activity_{i}"] == paths[i] for i in range(len_path)), axis=1)
  total = df['Found'].sum()
  sessions = df[df['Found'] == True]['SessionID'].nunique()
  return total,sessions


def all_paths_len_x(_df:pd.DataFrame,len):
  df = _df.copy()
  df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)
  # if the consecutive activity is the same as the current activity, remove the row
  # this is done because we only want to count transitions
  df = df[df["Activity"] != df["Consecutive_1"]]
  # remove the consecutive activity column
  df = df.drop("Consecutive_1", axis=1)
  # create a column for len_path - 1 consecutive activities
  df = df.assign(**{f"Activity_{i}": df.groupby("SessionID")["Activity"].shift(periods=-i) for i in range(1, len)})
  # change name of Activiy column to Activity_0
  df.rename(columns={'Activity':'Activity_0'}, inplace=True)
  # create a column that counts the amount of times the path is found
  df = df.groupby([f"Activity_{i}" for i in range(len)]).size().reset_index(name='count')
  total_count = df['count'].sum()
  df['probability'] = df['count'] / total_count
  return df
  
  

def activity_prob_df(_df:pd.DataFrame) -> pd.DataFrame:
  # group the dataframe by activity and count the number of times each activity occurs
  df = _df.copy()
  total_amount_of_activities = df['Activity'].count()
  # create df where each row contains the activity with columns: Activity, Count, Probability
  final_df = df.groupby('Activity').count().reset_index()
  # Drop column Timestamp
  final_df = final_df.drop('TIMESTAMP', axis=1)
  # change SessionID column name to Count
  final_df.rename(columns={'SessionID':'Count'}, inplace=True)
  # create Probability column
  final_df = final_df.assign(Probability=final_df['Count']/total_amount_of_activities * 100)
  
  return final_df.sort_values('Probability', ascending=False)
  


In [4]:
# Create df for pivot
df_for_pivot = create_df_for_pivot(base_df)
df_chances_1 = make_pivot(df_for_pivot, "level_0", "level_1")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = base_df.groupby("SessionID").nth(0)["Activity"].value_counts() / base_df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1.fillna(0, inplace=True)

df_chances_total = df_chances_1 
df_chances_total

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total["start_session_chance"][df_chances_total["start_session_chance"] > 0]

#Create df with all chances except the starting chance.
final_df = df_chances_total.iloc[:, :-1]
final_df
probability_matrix = final_df.copy()

In [5]:
activity_prob = activity_prob_df(base_df)
top_X = activity_prob.head(3)


In [6]:
l_2 = all_paths_len_x(base_df,2)
# drop probabilitie
l_2 = l_2.drop("probability", axis=1)
l_2

,Activity_0,Activity_1,count
0,/,/portal/pls/portal/PORTAL.wwsbr_javascript.pag...,1
1,/,/portal/pls/portal/PORTAL.wwsec_app_priv.logout,3
2,/,/shared/foutopgetreden.html,2
3,/,/werk_nl/werknemer/contact,1
4,/,/werk_nl/werknemer/home,125
...,...,...,...
15009,/zoeken_portlet/ajax/zoekBeroep,/xpsimage/wdo211812,1
15010,/zoeken_portlet/ajax/zoekBeroep,/xpsimage/wdo_010304,1
15011,/zoeken_portlet/ajax/zoekBeroep,/zoeken_portlet/ajax/zoekAantalIndicatief,9322
15012,/zoeken_portlet_wg/ajax/selectContactpersoon,/werk_nl/werkgever/cvs_zoeken,5


In [7]:
l_3 = all_paths_len_x(base_df,3)
# drop probabilitis
l_3 = l_3.drop("probability", axis=1)
l_3_no_c = l_3[l_3['Activity_0'] != l_3['Activity_2']]

In [8]:
l_3_no_c = l_3_no_c.sort_values('count', ascending=False)
l_3_final = l_3_no_c.head(1000)
l_3_final


,Activity_0,Activity_1,Activity_2,count
22098,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/home,17976
17217,/werk_nl/werknemer/home,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,17633
59357,/werk_nl/werknemer/werkmap,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,15372
59355,/werk_nl/werknemer/werkmap,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/home,14121
24057,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/home,13611
...,...,...,...,...
32760,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/over-werk-nl/handleiding/so...,187
21984,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/home,/werk_nl/werknemer/vacatures,187
60230,/werk_nl/werknemer/werkmap,/werk_nl/werknemer/over-werk-nl/handleiding,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,187
11520,/shared/timeout.htm,/werk_nl/werknemer/home,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/va...,187


In [9]:
# merge l_3_final with l_2 where l_3_final['Activity_0'] == l_2['Activity_0'] and l_3_final['Activity_2'] == l_2['Activity_1']
# keep columns Activity_0, Activity_1, Activity_2, count_x, count_y
merge = pd.merge(l_3_final, l_2, left_on=['Activity_0','Activity_2'], right_on=['Activity_0','Activity_1'], how='inner')

In [10]:
# drop probability_x	skip column
merge['facc'] = merge['count_x'] / merge['count_y']

merge = merge[merge['facc'] > 0]
merge = merge.sort_values('facc', ascending=False)

In [11]:
FAC = 15
merge_fac = merge[merge['facc'] > FAC]
# drop columns Activity_1_y
merge_f = merge_fac.drop("Activity_1_y", axis=1)
merge_f

,Activity_0,Activity_1_x,Activity_2,count_x,count_y,facc
671,/werk_nl/werknemer/uitkering-aanvragen/ww,/portal/page/portal/home/diensten/aanvragen-ww,/portal/page/portal/home/diensten/overzicht,728,1,728.000000
726,/werk_nl/werknemer/eintake/inschrijven,/portal/page/portal/home/diensten/aanvragen-ww,/portal/page/portal/home/diensten/overzicht,579,1,579.000000
847,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,/werk_nl/werknemer/solliciteren/ontwikkel-uzel...,/portal/page/portal/werk_nl/werknemer/sollicit...,344,1,344.000000
963,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_tips,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_t...,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_t...,216,2,108.000000
814,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/ewerkmap/readdetails,383,4,95.750000
535,/portal/page/portal/home/diensten/overzicht,/portal/page/portal/home/diensten/aanvragen-ww,/werk_nl/werknemer/home,1620,20,81.000000
817,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/va...,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/va...,/zoeken_portlet/ajax/zoekBeroep,377,6,62.833333
992,/zoeken_portlet/ajax/zoekAantalIndicatief,/werk_nl/werknemer/vacatures,/zoeken_portlet/ajax/addVacatureToGereageerd,189,4,47.250000
757,/werk_nl/werknemer/werkmap,/werk_nl/werknemer/vacatures,/zoeken_portlet/ajax/zoekBeroep,505,11,45.909091
950,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/over-werk-nl/handleiding/so...,/xpsitem/wdo_013142,225,6,37.500000


In [13]:
# skipping df = merge_f with columns Activity_0, Activity_2
skip_df = merge_f[['Activity_0', 'Activity_1_x','Activity_2']]
# change name of Activity_1_x to Activity_1

skip_df = skip_df.rename(columns={'Activity_1_x':'Activity_1'})


In [14]:
# Function to find the skipping patterns in a session.
def find_anomalous_sessions(df:pd.DataFrame,skipping_patterns:pd.DataFrame) -> list[int]:
  sessions_df = df.copy()
  # Create a new column with the consecutive activity
  sessions_df["Consecutive_Activity"] = sessions_df.groupby("SessionID")["Activity"].shift(periods=-1)  

  merged = pd.merge(sessions_df, skipping_patterns, left_on=['Activity','Consecutive_Activity'], right_on=['Activity_0','Activity_2'], how='inner',indicator='Anomaly')
  merged['Anomaly'] = np.where(merged.Anomaly == 'both', True, False)
  
  anomalous_session = merged[merged['Anomaly'] == True]
  # create a list with all SessionIDs that have an anomaly
  anomaly_sessions = anomalous_session['SessionID'].unique()
  
  return anomaly_sessions

In [15]:
sessions = find_anomalous_sessions(base_df,skip_df)


In [17]:
print(sessions)

[  114491  7750550 10034461 15536769 26494534 28708683 32890902 33597081
 34585045 36768748 39303671 44122579 44246297 44592614 50599136 50648210
 52144674 54593765   469014  3728991  8676664  8755382 17145900 23186071
 27707051 35447332   525807  1047249  3360767  3613894  7161329  8148792
 16540024 25742324 28905573 45862715 52071677   705041  4042470 12151497
 12418692 18138104 18406496 25336276 28720129 28802507 33168353 35546821
 37761106 41181760 45700775 48364807   747053  1057401  1171595  1176096
  2249993  3203332  3451055  5243194  5479088  7859875  9258077 10968312
 11362164 12127041 12220427 12273954 14811620 15625058 16861253 17449269
 17817523 18166469 18685381 19309642 20135221 20233803 20262110 21199706
 21628980 22728643 23645845 24631211 25398349 28437712 28621978 30256200
 30478274 34368837 35127156 39862671 40235372 40570582 42760838 43885456
 45385951 45905898 48166987 48491448 48968011 49907187 50267226 50985376
   827985  2707848  3616994  6003753  9034197 13120

In [19]:
# create a new column in the base_df to indicate if the session is anomalous
# if a session is anomalous it will have a True in the column else False
labeled_df = base_df.copy()
labeled_df["anomaly"] = base_df["SessionID"].apply(lambda x: x in sessions)

In [ ]:
labeled_df

,SessionID,TIMESTAMP,Activity,anomaly
3273278,46,2015-11-06 08:07:22.780,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
5333642,46,2015-11-06 08:07:40.767,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,False
3733243,46,2015-11-06 08:07:51.390,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,False
5904405,46,2015-11-06 08:08:06.003,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
5573282,46,2015-11-06 08:08:19.343,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
...,...,...,...,...
1613546,55314751,2016-02-28 08:17:15.947,/werk_nl/werknemer/werkmap,False
2925569,55314751,2016-02-28 08:18:10.877,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
4068602,55314751,2016-02-28 08:18:41.493,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
5013440,55314751,2016-02-28 08:19:59.737,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False


In [20]:
countAnomalies = labeled_df[labeled_df["anomaly"] == True]["SessionID"].nunique()
countAnomalies

355

In [22]:
countNormal = labeled_df[labeled_df["anomaly"] == False]["SessionID"].nunique()
countNormal

659915

In [23]:
countTotal= labeled_df["SessionID"].nunique()
countTotal

660270

In [24]:
countAnomalies + countNormal == countTotal

True

In [ ]:
# save in csv
# labeled_df.to_csv(f"./labeled_data/labeled_df_{FAC}.csv",index=False)
# skip_df.to_csv(f"./gen_patterns/skip_df_{FAC}.csv",index=False)